In [1]:
import tensorflow as tf

In [2]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3)
list(dataset.as_numpy_iterator())


[array([0, 1, 2], dtype=int64),
 array([3, 4, 5], dtype=int64),
 array([6, 7], dtype=int64)]

In [3]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3, drop_remainder=True)
list(dataset.as_numpy_iterator())


[array([0, 1, 2], dtype=int64), array([3, 4, 5], dtype=int64)]

In [16]:
sum(softmax)

<tf.Tensor: shape=(), dtype=float32, numpy=0.99999994>